In [3]:
##set up
import numpy as np
import pandas as pd
import string
import email
import os ##use this to traverse folders
from email.parser import Parser
import re
import lxml.html
import collections

In [4]:
#extract labels from TREC2007/full/ file lists
with open('trec07p/full/index', 'r') as ifile:
    raw_labels = ifile.readlines()

#using file list to give labels and e-mail numbers

categories = [i.split( )[0] for i in raw_labels]
email_nums = [i.split(".")[-1][:-1] for i in raw_labels]
labels = pd.Series(categories, index=email_nums)

In [5]:
#reference from other place

# parser = Parser()
# corpus = []
# for email_num in email_nums:
#     filepath = 'trec07p/data/inmail.%s' % email_num

#     with open(filepath, 'r',) as ifile:
#         email = parser.parse(ifile)

#     ## TODO also keep the subject header

#     ## code and details from:
#     ## https://stackoverflow.com/questions/17874360/python-how-to-parse-the-body-from-a-raw-email-given-that-raw-email-does-not/32840516#32840516
#     if email.is_multipart():
#         for part in email.walk():
#             content_type = part.get_content_type()
#             content_dispo = str(part.get('Content-Disposition'))

#             # skip any text/plain (txt) attachments
#             if content_type == 'text/plain' and 'attachment' not in content_dispo:
#                 body = part.get_payload(decode=False)
#                 break ## only keep the first email
#     else:
#         body = email.get_payload(decode=False)

#     ## TODO strip html stuff ? or valuable features ?
#     try:
#         corpus.append((email_num, body))
#     except NameError:
#         pass

# email_nums, corpus = zip(*corpus)

In [6]:
# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)

In [7]:
##encode
def robust_decode(bs):
    '''Takes a byte string as param and convert it into a unicode one.
First tries UTF8, and fallback to Latin1 if it fails'''
    cr = None
    try:
        #cr = bs.decode('utf8')
        cr = bs.decode('utf-8-sig')
    except UnicodeDecodeError:
        cr = bs.decode('ISO-8859-1')
        cr = cr.encode("ascii", "ignore")
        #cr = cr.decode('utf-8-sig')
    return cr

In [39]:
def info_of_email_2():
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    
    parser = Parser()
    info = []
    for nums in range(len(categories)):
        filepath = 'trec07p/data/inmail.%s' % email_nums[nums]
        
        with open(filepath, 'r') as ifile:
            email = parser.parse(ifile)
            
            Date = email.get("Date")
            Date = 

            Content = " ".join([robust_decode(i) for i in walkMsg(email) if i != None])
            Content = ''.join(Content.splitlines())
            Content = Content.replace("\t","")
            
            if 'html' in Content:
                Content = lxml.html.fromstring(Content).text_content()
            try: 
                year = re.search('\d{4}',Date).group(0)
            except:
                year = None
            try:
                hour = re.search('\d{2}:\d{2}:\d{2}',Date).group(0).split(":")[0]
            except:
                hour = None
            try: 
                weekday = re.search('Mon|Tue|Wed|Thu|Fri|Sat|Sun',Date).group(0)
            except: 
                weekday =None
            try:
                month = re.search('Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|Sept',Date).group(0)
            except:
                month =None
            info.append([year, weekday, hour, month,
                         email.get("From"),email.get('To'),
                         email.get('cc'),email.get('bcc'), 
                         str(email.get("Content-Type")).split(";")[0].lower(),
                         email.get('subject'),
                         Content,
                         categories[nums]])
    return info

In [40]:
df = pd.DataFrame(info_of_email_2(),
                 columns=['Year','Weekday',"Hour","Month",
                          'From','To','cc','bcc','Content_type','Subjects','Content','Classification'])

In [37]:
##need to figure out the way to deal with wrong year/mo/day....##
collections.Counter(df['Year'])

Counter({None: 1923,
         '0800': 17,
         '1900': 1,
         '1970': 3,
         '1980': 6,
         '1987': 8,
         '1988': 2,
         '1990': 39,
         '1994': 2,
         '1997': 2,
         '1998': 10,
         '1999': 11,
         '2000': 46,
         '2001': 100,
         '2002': 143,
         '2003': 272,
         '2004': 186,
         '2005': 105,
         '2006': 131,
         '2007': 72346,
         '2008': 23,
         '2009': 5,
         '2010': 1,
         '2011': 1,
         '2012': 1,
         '2013': 1,
         '2015': 3,
         '2017': 3,
         '2020': 1,
         '2025': 1,
         '2027': 2,
         '2029': 1,
         '2036': 1,
         '2038': 6,
         '2040': 1,
         '2080': 1,
         '2087': 2,
         '2090': 4,
         '2098': 1,
         '2099': 2,
         '2102': 2,
         '2761': 1,
         '4307': 1,
         '9744': 1})

In [56]:
collections.Counter(df['Month'])

Counter({None: 661,
         'Apr': 25371,
         'Aug': 46,
         'Dec': 46,
         'Feb': 103,
         'Jan': 300,
         'Jul': 4657,
         'Jun': 21951,
         'Mar': 136,
         'May': 22046,
         'Nov': 23,
         'Oct': 35,
         'Sep': 44})

In [73]:
filepath = 'trec07p/data/inmail.514'
parser = Parser()
with open(filepath, 'r') as ifile:
    email = parser.parse(ifile)

In [74]:
Date = email.get('Date')

In [75]:
print email

From freecall-w.com@lottozubotto.com  Mon Apr  9 04:52:21 2007
Return-Path: <freecall-w.com@lottozubotto.com>
Received: from localhost (eml2.internetdsl.tpnet.pl [83.15.171.2])
 by speedy.uwaterloo.ca (8.12.8/8.12.5) with SMTP id l398ps0I011899
 for <ktwarwic@speedy.uwaterloo.ca>; Mon, 9 Apr 2007 04:52:05 -0400
Message-ID: <000001c77a83$92782400$0100007f@localhost>
From: "Jeremy Bell" <freecall-w.com@lottozubotto.com>
To: <ktwarwic@speedy.uwaterloo.ca>
Subject: Play over 50 lotteries online
Date: Pn, 09 kwi 2007 10:51:45 +0200
MIME-Version: 1.0
Content-Type: multipart/related;
 boundary="----=_NextPart_000_0001_01C77A83.92782400"
X-Priority: 3
X-MSMail-Priority: Normal
X-Mailer: Microsoft Outlook Express 6.00.2600.0000
X-MimeOLE: Produced By Microsoft MimeOLE V6.00.2600.0000
Status: O
Content-Length: 19716
Lines: 310

This is a multi-part message in MIME format.

------=_NextPart_000_0001_01C77A83.92782400
Content-Type: multipart/alternative;
 boundary="----=_NextPart_001_000E_01C77A83

In [ ]:
#linearSVC
#tfidf (different type) + work2vec
#seperate year top 40

